In [ ]:
# import argparse
import sys
import os
import subprocess
from pathlib import Path
import xarray as xr
import numpy as np


In [1]:
"""
Complete example: Add TC parameters to tropical_cyclone_risk model output
and create visualizations.
"""

import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

import idd_climate_models.constants as rfc
# Import the parameter estimation functions
from idd_climate_models.add_tc_params import add_parameters_to_dataset, add_tc_parameters_to_track
from idd_climate_models.verify_params import plot_track_with_parameters, generate_dataset_summary


# =============================================================================
# STEP 1: Process your file to add parameters
# =============================================================================

print("="*70)
print("STEP 1: ADDING TC PARAMETERS TO YOUR FILE")
print("="*70)

# Your file path
input_file = '/mnt/team/rapidresponse/pub/tropical-storms/tc_risk/output/cmip6/ACCESS-CM2/r1i1p1f1/historical/1970-1989/SI/tracks_SI_ACCESS-CM2_historical_r1i1p1f1_197001_198912_e9.nc'


STEP 1: ADDING TC PARAMETERS TO YOUR FILE


In [2]:
ds = xr.open_dataset(input_file)

In [3]:
ds

<xarray.Dataset> Size: 11MB
Dimensions:          (n_trk: 220, time: 721, year: 20, basin: 7, month: 12)
Coordinates:
  * n_trk            (n_trk) int64 2kB 0 1 2 3 4 5 6 ... 214 215 216 217 218 219
  * time             (time) float64 6kB 0.0 3.6e+03 ... 2.588e+06 2.592e+06
  * year             (year) int64 160B 1970 1971 1972 1973 ... 1987 1988 1989
  * basin            (basin) <U2 56B 'AU' 'EP' 'NA' 'NI' 'SI' 'SP' 'WP'
  * month            (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Data variables: (12/13)
    lon_trks         (n_trk, time) float64 1MB ...
    lat_trks         (n_trk, time) float64 1MB ...
    u250_trks        (n_trk, time) float64 1MB ...
    v250_trks        (n_trk, time) float64 1MB ...
    u850_trks        (n_trk, time) float64 1MB ...
    v850_trks        (n_trk, time) float64 1MB ...
    ...               ...
    m_trks           (n_trk, time) float64 1MB ...
    vmax_trks        (n_trk, time) float64 1MB ...
    tc_month         (n_trk) float64 2kB ...
    tc_basins        (n_trk) <U2 2kB ...
    tc_years         (n_trk) int64 2kB ...
    seeds_per_month  (year, basin, month) float64 13kB ...

In [ ]:
# Read in a draw
# Prep TCTracks object
#  - Get number of tracks and timestep
#  - Loop over tracks to get fixed time
#  - Loop over tracks to get 

# def read_custom_tracks(
#     root_path: Path,
#     model: str,
#     variant: str,
#     scenario: str,
#     batch_year: str,
#     basin: str,
#     draw: int,
# ) -> TCTracks:
#     """
#     Read tc risk model outputs for CLIMADA processing.
#     """

# def prepare_minimal_tctracks_from_custom(ds_custom):
#     """
#     Convert custom synthetic tracks with time in seconds and per-track
#     year/month info into a TCTracks object compatible with
#     TropCyclone.from_tracks().
#     """




























from xarray.backends.file_manager import FILE_CACHE
import idd_climate_models.constants as rfc
from idd_climate_models.climate_file_functions import is_curvilinear_grid, regrid_with_cdo, recombine_variable_files
from idd_climate_models.validation_functions import is_monthly, filter_files_by_time_period

def main():
    parser = argparse.ArgumentParser(description='Process a single variable/frequency combination')
    parser.add_argument('--data_source', required=True)
    parser.add_argument('--model', required=True)
    parser.add_argument('--variant', required=True)
    parser.add_argument('--scenario', required=True)
    parser.add_argument('--time_period', required=True)
    parser.add_argument('--variable', required=True)
    parser.add_argument('--grid', required=True)
    parser.add_argument('--frequency', required=True)
    parser.add_argument('--needs_regridding_str', required=True, help='String that indicates if regridding is needed')

    args = parser.parse_args()
    args.needs_regridding = args.needs_regridding_str.lower() == 'true'

    # --- Directory setup ---
    processed_data_path = Path(rfc.PROCESSED_DATA_PATH)
    target_base_path = rfc.TC_RISK_INPUT_PATH / args.data_source

    source_dir = processed_data_path / args.data_source / args.model / args.variant / args.scenario / args.variable / args.grid / args.frequency
    target_dir = target_base_path / args.model / args.variant / args.scenario / args.time_period
    
    print(f"Processing {args.variable} for {target_dir}")
    
    target_dir.mkdir(parents=True, exist_ok=True)
    
    # --- File finding and combining logic ---
    combined_files = 0
    errors = []
    
    bin_name = args.time_period
    bin_start_str, bin_end_str = args.time_period.split('-')
    bin_start = int(bin_start_str)
    bin_end = int(bin_end_str)
    
    if is_monthly(args.frequency):
        date_start = f"{bin_start}01" 
        date_end = f"{bin_end}12" 
    else:
        date_start = f"{bin_start}0101" 
        date_end = f"{bin_end}1231" 
    
    files_in_range = filter_files_by_time_period(
        str(source_dir), args.variable, bin_start, bin_end, args.frequency,
        args.model, args.scenario, args.variant, args.grid
    )
    
    if files_in_range:
        # Check if files need regridding to determine output grid label
        needs_regridding = args.needs_regridding
        output_grid = 'gr' if needs_regridding else args.grid
        
        output_filename = f"{args.variable}_{args.frequency}_{args.model}_{args.scenario}_{args.variant}_{output_grid}_{date_start}-{date_end}.nc"
        output_path = target_dir / output_filename
        
        print(f"  → Combining {len(files_in_range)} files...")
        if needs_regridding:
            print(f"  → Output grid will be: {output_grid} (regridded from {args.grid})")
        
        success, was_regridded = recombine_variable_files(
            sorted(set(files_in_range)), output_path, args.variable
        )
        
        if success:
            combined_files += 1
            print(f"  ✅ Combined → {output_filename}")
        else:
            error_msg = f"Failed to combine {args.variable} files for {bin_name}" 
            errors.append(error_msg)
            print(f"  ❌ {error_msg}")
    else:
        print(f"  ⚠️  No files found for {args.variable} in {bin_start}-{bin_end}")
    
    print(f"\n✅ Completed {args.variable}/{args.frequency}")
    print(f"   Combined files: {combined_files}")
    
    if errors:
        print(f"   Errors: {len(errors)}")
        sys.exit(1)


if __name__ == "__main__":
    main()

In [1]:
from climada.hazard import TCTracks, TropCyclone, Centroids  # type: ignore
import xarray as xr  # type: ignore
import numpy as np # type: ignore
import re
from pathlib import Path
import os
import datetime as dt
import argparse

def normalize_lon(lon: float) -> float:
    """Normalize longitude to [-180, 180] range."""
    lon = ((lon + 180) % 360) - 180
    return lon

def generate_basin_centroids(basin: str, res: float = 0.1) -> "Centroids":
    """
    Generate Centroids for a specific tropical cyclone basin.
    """

    # Dictionary of basin bounds
    basin_bounds = {
        'EP': ['180E', '0N', '290E', '60N'],
        'NA': ['260E', '0N', '360E', '60N'],
        'NI': ['30E', '0N', '100E', '50N'],
        'SI': ['20E', '45S', '100E', '0S'],
        'AU': ['100E', '45S', '180E', '0S'],
        'SP': ['180E', '45S', '250E', '0S'], # Original SA - possible mismatch
        'WP': ['100E', '0N', '180E', '60N'],
        'GL': ['0E', '90S', '360E', '90N']
    }

    if basin not in basin_bounds:
        raise ValueError(f"Basin '{basin}' not recognized. Available: {list(basin_bounds.keys())}")

    def parse_coord(coord_str: str) -> float:
        """Convert coordinate string with direction to float degrees."""
        match = re.match(r"([0-9\.]+)([ENWS])", coord_str)
        if not match:
            raise ValueError(f"Invalid coordinate string: {coord_str}")
        val, dir_ = match.groups()
        val = float(val)
        if dir_ in ['W', 'S']:
            val = -val
        return val

    lon_min, lat_min, lon_max, lat_max = [parse_coord(c) for c in basin_bounds[basin]]

    # Normalize longitudes to [-180, 180]
    lon_min = normalize_lon(lon_min)
    lon_max = normalize_lon(lon_max)

    # Expand upper bounds by resolution to include last grid cell
    lon_max += res
    lat_max += res

    # Create Centroids for the basin
    centroids = Centroids.from_pnt_bounds((lon_min, lat_min, lon_max, lat_max), res=res)

    return centroids

ModuleNotFoundError: No module named 'climada'